# InfluxDB

---

## Introduction

### Install Dependencies
First, you need to install the influxdb-client module. Run the command below in your terminal.

### Tokens
InfluxDB Cloud uses Tokens to authenticate API access. We've created an all-access token for you for this set up process.

Run this command in your terminal to save your token as an environment variable:

export INFLUXDB_TOKEN="your_token"

### Initialize Client

In [1]:
_org = 'your_org'
_url = 'your_url'
_token = 'your_influx_token'

In [2]:
import influxdb_client, os, time
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS

token = _token  # OR os.environ.get("INFLUXDB_TOKEN")
org = _org
url = _url

client = influxdb_client.InfluxDBClient(url=url, token=token, org=org)

### Write Data

To start writing data, we need a place to store our time-series data. We call these buckets.  

In [3]:
# TEST BUCKET NAME: test
bucket="test"

write_api = client.write_api(write_options=SYNCHRONOUS)
   
for value in range(5):
    point = (
        Point("measurement1")
        .tag("tagname1", "tagvalue1")
        .field("field1", value)
    )
    write_api.write(bucket=bucket, org="fbaquant", record=point)
    time.sleep(1) # separate points by 1 second

## In the above code snippet, we define five data points and write each one to InfluxDB.  

Each of the 5 points we write has a field and a tag.

### Execute a Flux Query

Now let’s query the numbers we wrote into the database.  
We use the Flux scripting language to query data.  
Flux is designed for querying, analyzing, and acting on data.

In this query, we are looking for data points within the last 10 minutes with a measurement of "measurement1".  

Let’s use that Flux query in our Python code!Run the following:

In [4]:
import pandas as pd

query_api = client.query_api()

query = """from(bucket: "test")
 |> range(start: -1m)
 |> filter(fn: (r) => r._measurement == "measurement1")"""
tables = query_api.query(query, org=_org)

data = []
for table in tables:
    for record in table.records:
        data.append(record.values)

pd.DataFrame(data)

,result,table,_start,_stop,_time,_value,_field,_measurement,tagname1
0,_result,0,2025-04-16 02:17:23.858308+00:00,2025-04-16 02:18:23.858308+00:00,2025-04-16 02:18:18.039678+00:00,0,field1,measurement1,tagvalue1
1,_result,0,2025-04-16 02:17:23.858308+00:00,2025-04-16 02:18:23.858308+00:00,2025-04-16 02:18:19.187896+00:00,1,field1,measurement1,tagvalue1
2,_result,0,2025-04-16 02:17:23.858308+00:00,2025-04-16 02:18:23.858308+00:00,2025-04-16 02:18:20.316103+00:00,2,field1,measurement1,tagvalue1
3,_result,0,2025-04-16 02:17:23.858308+00:00,2025-04-16 02:18:23.858308+00:00,2025-04-16 02:18:21.413889+00:00,3,field1,measurement1,tagvalue1
4,_result,0,2025-04-16 02:17:23.858308+00:00,2025-04-16 02:18:23.858308+00:00,2025-04-16 02:18:22.537171+00:00,4,field1,measurement1,tagvalue1


### Execute an Aggregate Query

Aggregate functions take the values of all rows in a table and use them to perform an aggregate operation.  

In this example, we use the mean() function to calculate the average value of data points in the last 10 minutes.

In [5]:
query_api = client.query_api()

query = """from(bucket: "test")
  |> range(start: -10m)
  |> filter(fn: (r) => r._measurement == "measurement1")
  |> mean()"""
tables = query_api.query(query, org="fbaquant")

for table in tables:
    for record in table.records:
        print(record)

FluxRecord() table: 0, {'result': '_result', 'table': 0, '_start': datetime.datetime(2025, 4, 16, 2, 8, 23, 888996, tzinfo=datetime.timezone.utc), '_stop': datetime.datetime(2025, 4, 16, 2, 18, 23, 888996, tzinfo=datetime.timezone.utc), '_value': 2.0, '_field': 'field1', '_measurement': 'measurement1', 'tagname1': 'tagvalue1'}


---

### Advanced: Write Data with Batch

In [6]:
import time
from datetime import datetime as dt

from influxdb_client import WriteOptions, WritePrecision

bucket="test"

write_api = client.write_api(
    write_options=WriteOptions(batch_size=1000, flush_interval=1000)  # flush_interval: ms
)

for value in range(10_000):
    point = (
        Point("measurement2")
        .tag("tagname1", "tagvalue1")
        .field("field1", value)
        .time(time.time_ns())
    )
    time.sleep(0.0001)
    write_api.write(bucket=bucket, org=_org, record=point)
    
write_api.close()

In [7]:
import pandas as pd

query_api = client.query_api()

query = """from(bucket: "test")
 |> range(start: -1m)
 |> filter(fn: (r) => r._measurement == "measurement2")"""
tables = query_api.query(query, org=_org)

data = []
for table in tables:
    for record in table.records:
        data.append(record.values)

print(pd.DataFrame(data).shape)
pd.DataFrame(data).head(10)

(10000, 9)


,result,table,_start,_stop,_time,_value,_field,_measurement,tagname1
0,_result,0,2025-04-16 02:17:29.458379+00:00,2025-04-16 02:18:29.458379+00:00,2025-04-16 02:18:23.904963+00:00,0,field1,measurement2,tagvalue1
1,_result,0,2025-04-16 02:17:29.458379+00:00,2025-04-16 02:18:29.458379+00:00,2025-04-16 02:18:23.907107+00:00,1,field1,measurement2,tagvalue1
2,_result,0,2025-04-16 02:17:29.458379+00:00,2025-04-16 02:18:29.458379+00:00,2025-04-16 02:18:23.908163+00:00,2,field1,measurement2,tagvalue1
3,_result,0,2025-04-16 02:17:29.458379+00:00,2025-04-16 02:18:29.458379+00:00,2025-04-16 02:18:23.908718+00:00,3,field1,measurement2,tagvalue1
4,_result,0,2025-04-16 02:17:29.458379+00:00,2025-04-16 02:18:29.458379+00:00,2025-04-16 02:18:23.909250+00:00,4,field1,measurement2,tagvalue1
5,_result,0,2025-04-16 02:17:29.458379+00:00,2025-04-16 02:18:29.458379+00:00,2025-04-16 02:18:23.909777+00:00,5,field1,measurement2,tagvalue1
6,_result,0,2025-04-16 02:17:29.458379+00:00,2025-04-16 02:18:29.458379+00:00,2025-04-16 02:18:23.910306+00:00,6,field1,measurement2,tagvalue1
7,_result,0,2025-04-16 02:17:29.458379+00:00,2025-04-16 02:18:29.458379+00:00,2025-04-16 02:18:23.910831+00:00,7,field1,measurement2,tagvalue1
8,_result,0,2025-04-16 02:17:29.458379+00:00,2025-04-16 02:18:29.458379+00:00,2025-04-16 02:18:23.911359+00:00,8,field1,measurement2,tagvalue1
9,_result,0,2025-04-16 02:17:29.458379+00:00,2025-04-16 02:18:29.458379+00:00,2025-04-16 02:18:23.911883+00:00,9,field1,measurement2,tagvalue1


---